In [1]:
import os
import re
import warnings

import camelot
import pandas as pd
import pdfplumber
from PyPDF2 import PdfReader

warnings.filterwarnings("ignore")

In [2]:
# Function to extract table from a single PDF
def extract_bsn_table_from_pdf(pdf_path):
    try:
        # print(f"\nExtracting BSN table from '{os.path.basename(pdf_path)}'...")
        tables = camelot.read_pdf(pdf_path, pages="1", flavor="lattice")
        if tables:
            return tables[0].df  # Return the first table as a DataFrame
        else:
            print(f"\nNo tables found in '{pdf_path}'.")
            return None
    except Exception as e:
        print(f"\nError processing '{pdf_path}': {e}")
        return None


# Function to extract "Service Description ID" from a table
def extract_bsn_number_from_table(table_df):
    if table_df is not None:
        table_df.columns = ["Name", "Value"]  # Rename columns
        table_df = table_df.map(lambda x: str(x).replace(" ", "").strip() if pd.notna(x) else x)  # Remove extra spaces
        if "ServiceDescriptionID:" in table_df["Name"].values:
            bsn_value = table_df.loc[table_df["Name"] == "ServiceDescriptionID:", "Value"].values[0]

            # Check if the value already starts with 'BSN'
            if not bsn_value.startswith("BSN"):
                bsn_value = "BSN" + bsn_value

            print(f"Extracted BSN Number: {bsn_value}")
            return bsn_value
        else:
            print("Service Description ID not found in table.\n")
    return None


# Function to extract "Service Description ID" from a table
def extract_version_number_from_table(table_df):
    version_match = "v01.40"
    value = False
    if table_df is not None:
        table_df.columns = ["Name", "Value"]  # Rename columns
        table_df = table_df.map(str.strip)  # Remove leading/trailing spaces

        if "SD Template Version:" in table_df["Name"].values:
            version_number = table_df.loc[table_df["Name"] == "SD Template Version:", "Value"].values[0]

            # Check if the SD file is in new version
            if version_number == version_match:
                value = True
        else:
            print("\nSD Version is not found")

    return value

In [ ]:
# Function to find the run of service text
def find_ros_page_number(pdf_path, search_text):
    occurrences = []  # List to store all occurrences
    compiled_pattern = re.compile(search_text, re.IGNORECASE)

    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page_number, page in enumerate(pdf.pages, start=1):  # Pages are 1-indexed
                text = page.extract_text()
                if text:  # Ensure the page contains text
                    normalized_text = re.sub(r'\s+', ' ', text.strip())  # Normalize text

                    if compiled_pattern.search(normalized_text):
                        occurrences.append(page_number)  # Store the page number

        # If no occurrences found, return an empty list
        if not occurrences:
            print(f"\nNo occurrences of '{search_text}' found in '{os.path.basename(pdf_path)}'.")

    except Exception as e:
        print(f"\nError processing '{os.path.basename(pdf_path)}': {e}")

    return occurrences  # Return the full list of occurrences


def extract_ros_details_from_pdf(pdf_path):


In [3]:
def main(pdf_path, excel_path):
    ### Main code to extract BSN details
    bsn_table_df = extract_bsn_table_from_pdf(pdf_path)
    bsn_value = extract_bsn_number_from_table(bsn_table_df)
    # version_number = extract_version_number_from_table(bsn_table_df)
    #----------------------------------------------------------------------------------------------------------------------#

    ### Main code to extract Run of Service
    print("\nThe Run of Service details:")
    print("-------------------------")

    empty_dict = {"": ""}

    ros_start_text = r"\d+\.\d+(\.\d+)?\sRun of Service"
    ros_end_text = r"\d+\.\d+(\.\d+)?\sRetirement of Service"

    start_pages, end_pages, material_page_numbers = create_page_list(pdf_path, material_search_start_text,
                                                                     material_search_end_text)

    # print(f"Found Start Pages: {start_pages}")
    # print(f"Found End Pages: {end_pages}")
    # print(f"Page Range: {material_page_numbers}")

    if material_page_numbers:
        extracted_material_data = extract_data_from_material_tables(pdf_path, material_page_numbers)
        if len(extracted_material_data) == 0:
            print("\nService Availability details are not available in the SD file.\n")
            print(empty_dict)
        else:
            print(f"\n{extracted_material_data}")
    else:
        print("\nService Availability details are not available in the SD file.")
    #----------------------------------------------------------------------------------------------------------------------#

    # ### Main code to insert all the extracted data into the Excel
    # print("\n\nGetting data into the Excel...\n")
    #
    # insert_data_to_excel(excel_path, bsn_value, response_time_list, resolution_time_list, extracted_material_data,
    #                      extracted_drc_value, support_hour_dict)

In [4]:
# Define paths
folder_path = r"C:/Users/rmya5fe/OneDrive - Allianz/01_Automated Reports/07_Sample_SDs"
database_path = os.path.join(folder_path, "Database")
excel_path = os.path.join(folder_path, "01_SLA_extract_from_SD.xlsx")
# excel_path = os.path.join(folder_path, "Database\_sample_sd\SLA_extract_from_SD.xlsx")

# Example usage
file_name = "8000013_FILE_SERVICE_LINE_FILEXCHANGE_SD_V04.10.pdf"  # Example file name
pdf_path = os.path.join(database_path, file_name)
main(pdf_path, excel_path)

Extracted BSN Number: BSN8000013


In [ ]:
# Different pdf
# 8033151